1) Зчитування dotenv

In [319]:
import os
import imaplib
import email
import xml.etree.ElementTree as ET
import requests
from dotenv import load_dotenv
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import decode_header
import shutil
from collections import defaultdict

load_dotenv(override=True)

API_KEY = os.getenv("GOOGLE_API_KEY")
START_LAT = os.getenv("START_LAT")
START_LON = os.getenv("START_LON")
START_COORD = f"{START_LAT},{START_LON}"
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_PASS = os.getenv("EMAIL_PASS")
IMAP_SERVER = os.getenv("IMAP_SERVER")


2) Зчитування файлу

In [320]:
def connect_to_mail():
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL_USER, EMAIL_PASS)
    mail.select("inbox")
    return mail

def decode_subject(subject_raw):
    decoded_fragments = decode_header(subject_raw)
    subject = ""
    for fragment, encoding in decoded_fragments:
        if isinstance(fragment, bytes):
            subject += fragment.decode(encoding or "utf-8", errors="ignore")
        else:
            subject += fragment
    return subject.strip()

def fetch_xml_from_mail():
    print("📬 Підключення до пошти...")

    mail = connect_to_mail()

    status, messages = mail.search(None, 'UNSEEN')
    mail_ids = messages[0].split()
    print(f"📨 Непрочитані листи: {len(mail_ids)}")

    results = []

    for num in mail_ids:
        print(f"\n🔍 Перевіряю лист № {num.decode()}")
        _, msg_data = mail.fetch(num, "(RFC822)")
        message = email.message_from_bytes(msg_data[0][1])

        sender = email.utils.parseaddr(message["From"])[1]
        subject = decode_subject(message["Subject"] or "Без теми")
        print(f"➡️ Від: {sender}")
        print(f"📝 Тема: {subject}")

        for part in message.walk():
            filename = part.get_filename()
            maintype = part.get_content_maintype()

            if filename and filename.endswith(".xml"):
                print(f"📎 Знайдено вкладення: {filename} (тип: {maintype})")

                os.makedirs("temp", exist_ok=True)
                filepath = os.path.join("temp", filename)

                with open(filepath, "wb") as f:
                    f.write(part.get_payload(decode=True))

                print(f"✅ Файл збережено: {filepath}")
                results.append((filepath, sender, subject))
                # break  # тільки один XML з листа

    mail.logout()

    if not results:
        print("📭 Завершено. Немає листів з XML-вкладенням.")
        return []

    return results

def parse_xml(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    locations = []

    for loc in root.findall("Location"):
        name = loc.find("Name").text
        lat  = float(loc.find("Latitude").text)
        lon  = float(loc.find("Longitude").text)
        locations.append((name, lat, lon))

    return locations

3) Підключення до API та отримання відстані

In [321]:
def get_distance_matrix(all_points, matrix):

    n = len(all_points)
    
    for i in range(n):
        origin_coord = f"{all_points[i][1]},{all_points[i][2]}"
        destination_coords = [f"{point[1]},{point[2]}" for point in all_points]

        url = "https://maps.googleapis.com/maps/api/distancematrix/json"
        params = {
            "origins": origin_coord,
            "destinations": "|".join(destination_coords),
            "key": API_KEY,
            "units": "metric",
            "mode": "driving",
        }

        response = requests.get(url, params=params)
        data = response.json()

        if data["status"] != "OK":
            print(f"❌ Запит API не спрацював для точки {all_points[i][0]}")
            continue

        elements = data["rows"][0]["elements"]
        for j in range(n):
            element = elements[j]
            if element["status"] == "OK":
                matrix[i][j] = element["distance"]["value"]  # у метрах
            else:
                matrix[i][j] = float("inf")  # недоступна точка

    return all_points, matrix


4) Побудова повної матриці

In [322]:
def build_full_matrix_with_start(locations):

    all_points = [("START", float(START_COORD.split(",")[0]), float(START_COORD.split(",")[1]))] + locations
    
    matrix = [[0.0] * len(all_points) for _ in range(len(all_points))]
    
    return get_distance_matrix(all_points, matrix)


5) Оптимізація маршруту (найближчий сусід)

In [323]:
def tsp_nearest_neighbor(matrix, start_index=0):
    n = len(matrix)
    visited = [False] * n
    path = [start_index]
    visited[start_index] = True
    current = start_index

    for _ in range(n - 1):
        next_city = None
        min_dist = float("inf")
        for i in range(n):
            if not visited[i] and matrix[current][i] < min_dist:
                min_dist = matrix[current][i]
                next_city = i

        path.append(next_city)
        visited[next_city] = True
        current = next_city

    path.append(start_index)
    return path

def extract_route_info(route, all_points, matrix):
    info = []

    for i in range(len(route) - 1):
        from_idx = route[i]
        to_idx = route[i + 1]

        name = all_points[to_idx][0]
        dist_m = matrix[from_idx][to_idx]
        dist_km = f"{dist_m / 1000:.1f} km" if dist_m != float("inf") else "—"
        dur_min = f"{round(dist_m / 800)} mins" if dist_m != float("inf") else "—"

        info.append((name, dist_km, dur_min))

    return info


6) Генерування посилання

In [324]:
def format_full_distance_table(all_points, matrix):
    headers = [name for name, _, _ in all_points]
    lines = []

    # Заголовок
    header_line = f"{'':<12}" + "".join([f"{h:<12}" for h in headers])
    lines.append(header_line)
    lines.append("-" * len(header_line))

    # Рядки
    for i, (from_name, _, _) in enumerate(all_points):
        row = f"{from_name:<12}"
        for j in range(len(all_points)):
            dist_m = matrix[i][j]
            dist_km = f"{dist_m / 1000:.1f} km" if dist_m != float("inf") else "—"
            row += f"{dist_km:<12}"
        lines.append(row)

    return "\n".join(lines)

def generate_maps_url(route, all_points):
    coords = []

    for idx in route:
        lat = all_points[idx][1]
        lon = all_points[idx][2]
        coords.append(f"{lat},{lon}")

    # 🧭 Перевіряємо: чи маршрут вже закінчується на START?
    if route[-1] != 0:
        coords.append(f"{all_points[0][1]},{all_points[0][2]}")

    url = "https://www.google.com/maps/dir/" + "/".join(coords)
    return url


7) Відправка результату

In [325]:
def send_result_email(recipient_email, original_subject, body_text):
    subject = f"Найкоротший маршрут: {original_subject}"

    message = MIMEMultipart()
    message["From"] = EMAIL_USER
    message["To"] = recipient_email
    message["Subject"] = subject
    message.attach(MIMEText(body_text, "plain"))

    server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    server.login(EMAIL_USER, EMAIL_PASS)
    server.send_message(message)
    server.quit()

    print(f"📤 Відповідь надіслано на {recipient_email} з темою: '{subject}'")


8) Очистка папки temp

In [326]:
def cleanup_temp_xml_files():
    temp_dir = "temp"
    for filename in os.listdir(temp_dir):
        if filename.endswith(".xml"):
            os.remove(os.path.join(temp_dir, filename))
            print(f"🧺 Видалено файл: {filename}")


9) Запуск усього процесу

In [328]:
messages = fetch_xml_from_mail()

    # 📦 Групуємо файли по листах
grouped = defaultdict(list)
for filepath, sender, subject in messages:
    grouped[(sender, subject)].append(filepath)

for (sender, subject), xml_files in grouped.items():
    
    body = f"📬 Ви надіслали {len(xml_files)} XML-файл(ів).\n\n"
    body += f"📦 Нижче наведено маршрути для кожного з них:\n\n"
    
    for i, xml_file in enumerate(xml_files, start=1):
        
        body += "📊 Відстані між точками:\n\n" + full_table + "\n\n"
        body += f"🧭 Маршрут №{i}: {os.path.basename(xml_file)}\n\n"

        try:
            delivery_points = parse_xml(xml_file)
            all_points, matrix = build_full_matrix_with_start(delivery_points)
            route = tsp_nearest_neighbor(matrix)
            route_info = extract_route_info(route, all_points, matrix)
            maps_url = generate_maps_url(route, all_points)
            full_table = format_full_distance_table(all_points, matrix)

            body += f"{'Точка':<25} {'Відстань':<12} {'Час в дорозі'}\n"
            body += "-" * 55 + "\n"
            for name, distance, duration in route_info:
                body += f"{name:<25} {distance:<12} {duration}\n"

            body += f"\n🗺️ Візуалізувати маршрут: {maps_url}\n"

        except Exception as e:
            body += f"\n⚠️ Не вдалося обробити файл {os.path.basename(xml_file)}: {e}\n"

        body += "\n" + "=" * 60 + "\n\n"

    send_result_email(sender, subject, body)
cleanup_temp_xml_files()

📬 Підключення до пошти...
📨 Непрочитані листи: 2

🔍 Перевіряю лист № 3
➡️ Від: peca.sasha@gmail.com
📝 Тема: Test 3
📎 Знайдено вкладення: test2.xml (тип: text)
✅ Файл збережено: temp\test2.xml

🔍 Перевіряю лист № 4
➡️ Від: peca.sasha@gmail.com
📝 Тема: Test 4
📎 Знайдено вкладення: test2.xml (тип: text)
✅ Файл збережено: temp\test2.xml
📎 Знайдено вкладення: test.xml (тип: text)
✅ Файл збережено: temp\test.xml
📤 Відповідь надіслано на peca.sasha@gmail.com з темою: 'Найкоротший маршрут: Test 3'
📤 Відповідь надіслано на peca.sasha@gmail.com з темою: 'Найкоротший маршрут: Test 4'
🧺 Видалено файл: test.xml
🧺 Видалено файл: test2.xml
